PyTorch Dataset: https://pytorch.org/tutorials/beginner/data_loading_tutorial.html

NOTE:
1. The validation range is changed to small size for debug.
2. Mean:  [0.90960454 0.81946206 0.87811487]
   Std:  [0.13244118 0.24944844 0.16392948]

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F # stateless functions
from torch.utils.data import Dataset, DataLoader
from torch.utils.data import sampler

import os
import pandas as pd
from skimage import io, transform
import numpy as np
import copy

import matplotlib.pyplot as plt

#import torchvision.datasets as dset
import torchvision.transforms as T
import torchvision.models as models

import time
from collections import defaultdict

In [3]:
BASE_FOLDER = "/project/data/"
!ls {BASE_FOLDER}


train = pd.read_csv(os.path.join(BASE_FOLDER, 'train.csv'))
test = pd.read_csv(os.path.join(BASE_FOLDER, 'test.csv'))

image_dir = os.path.join(BASE_FOLDER, 
                         'train_images')
mask_dir = os.path.join(BASE_FOLDER,
                       'train_label_masks')

print(train[train['gleason_score']=='4+3'].groupby(['isup_grade']).size())

query = (train['gleason_score'] == "4+3") & (train['isup_grade'] ==2)
print(train.index[query])

train.drop(train.index[query], inplace=True)

train["gleason_score"] = train["gleason_score"].apply(lambda x : "0+0" if x=="negative" else x)

g0_r = train[(train['gleason_score']=="0+0") & (train['data_provider'] == 'radboud')].sample(5).index
g0_k = train[(train['gleason_score']=="0+0") & (train['data_provider'] == 'karolinska')].sample(5).index

g1_r = train[(train['gleason_score']=="3+3") & (train['data_provider'] == 'radboud')].sample(5).index
g1_k = train[(train['gleason_score']=="3+3") & (train['data_provider'] == 'karolinska')].sample(5).index

g2_r = train[(train['gleason_score']=="3+4") & (train['data_provider'] == 'radboud')].sample(5).index
g2_k = train[(train['gleason_score']=="3+4") & (train['data_provider'] == 'karolinska')].sample(5).index

g3_r = train[(train['gleason_score']=="4+3") & (train['data_provider'] == 'radboud')].sample(5).index
g3_k = train[(train['gleason_score']=="4+3") & (train['data_provider'] == 'karolinska')].sample(5).index

# 15 for radboud isup 4 : 4+4, 3+5 , 5+3
g4_44_r = train[(train['gleason_score']=="4+4") & (train['data_provider'] == 'radboud')].sample(5).index
g4_35_r = train[(train['gleason_score']=="3+5") & (train['data_provider'] == 'radboud')].sample(5).index
g4_53_r = train[(train['gleason_score']=="5+3") & (train['data_provider'] == 'radboud')].sample(5).index

# 15 for karolinska isup 4 : 4+4, 3+5 , 5+3
g4_44_k = train[(train['gleason_score']=="4+4") & (train['data_provider'] == 'karolinska')].sample(5).index
g4_35_k = train[(train['gleason_score']=="3+5") & (train['data_provider'] == 'karolinska')].sample(5).index
g4_53_k = train[(train['gleason_score']=="5+3") & (train['data_provider'] == 'karolinska')].sample(2).index # we have we few samples here

# 15 for radboud isup 5 : 4+5, 5+4 , 5+5
g5_45_r = train[(train['gleason_score']=="4+5") & (train['data_provider'] == 'radboud')].sample(5).index
g5_54_r = train[(train['gleason_score']=="5+4") & (train['data_provider'] == 'radboud')].sample(5).index
g5_55_r = train[(train['gleason_score']=="5+5") & (train['data_provider'] == 'radboud')].sample(5).index

# 15 for karolinska isup 5 : 4+5, 5+4 , 5+5
g5_45_k = train[(train['gleason_score']=="4+5") & (train['data_provider'] == 'karolinska')].sample(5).index
g5_54_k = train[(train['gleason_score']=="5+4") & (train['data_provider'] == 'karolinska')].sample(5).index
g5_55_k = train[(train['gleason_score']=="5+5") & (train['data_provider'] == 'karolinska')].sample(5).index

indices_samples = g0_r | g0_k |  g1_r | g1_k | g2_r | g2_k | g3_r | g3_k | g4_44_r | g4_35_r | g4_53_r | g4_44_k | g4_35_k | g4_53_k | g5_45_r | g5_54_r | g5_55_r | g5_45_k  | g5_54_k | g5_55_k

print("Collected small samples" , len(indices_samples))

small_train = train.loc[indices_samples]

prostate-cancer-grade-assessment.zip  test.csv	 train_images
sample_submission.csv		      train.csv  train_label_masks
isup_grade
2       1
3    1242
dtype: int64
Int64Index([7273], dtype='int64')
Collected small samples 97


In [24]:
from os.path import splitext
from os import listdir
import numpy as np
from glob import glob
import torch
from torch.utils.data import Dataset
import logging
from PIL import Image
import skimage.io

listdir(image_dir)
ids = [splitext(file)[0] for file in listdir(image_dir)
                    if  file.endswith('.tiff')]
print(len(ids))


image = skimage.io.MultiImage(os.path.join(image_dir,ids[0]+".tiff"))
image = np.array(os.path.join(image_dir,ids[0]+".tiff"))
print(os.path.join(image_dir,ids[0]+".tiff"))


10616
/project/data/train_images/1e1862f8927813cacf7391cc55848ba4.tiff


In [ ]:


class BasicDataset(Dataset):
    def __init__(self, imgs_dir, masks_dir, scale=1):
        self.imgs_dir = imgs_dir
        self.masks_dir = masks_dir
        self.scale = scale
        assert 0 < scale <= 1, 'Scale must be between 0 and 1'

        self.ids = [splitext(file)[0] for file in listdir(imgs_dir)
                    if not file.endswith('.tiff')]
        logging.info(f'Creating dataset with {len(self.ids)} examples')

    def __len__(self):
        return len(self.ids)

    @classmethod
    def preprocess(cls, pil_img, scale):
        w, h = pil_img.size
        newW, newH = int(scale * w), int(scale * h)
        assert newW > 0 and newH > 0, 'Scale is too small'
        pil_img = pil_img.resize((newW, newH))

        img_nd = np.array(pil_img)

        if len(img_nd.shape) == 2:
            img_nd = np.expand_dims(img_nd, axis=2)

        # HWC to CHW
        img_trans = img_nd.transpose((2, 0, 1))
        if img_trans.max() > 1:
            img_trans = img_trans / 255

        return img_trans

    def __getitem__(self, i):
        
        image = skimage.io.MultiImage(os.path.join())[-2]
        image = np.array(image)

        mask = skimage.io.MultiImage(slide_path)[-2]
        mask = np.array(image)

        idx = self.ids[i]
        mask_file = glob(self.masks_dir + idx + '*')
        img_file = glob(self.imgs_dir + idx + '*')
        assert len(mask_file) == 1, \
            f'Either no mask or multiple masks found for the ID {idx}: {mask_file}'
        assert len(img_file) == 1, \
            f'Either no image or multiple images found for the ID {idx}: {img_file}'
        mask = Image.open(mask_file[0])
        img = Image.open(img_file[0])

        assert img.size == mask.size, \
            f'Image and mask {idx} should be the same size, but are {img.size} and {mask.size}'

        img = self.preprocess(img, self.scale)
        mask = self.preprocess(mask, self.scale)

        return {'image': torch.from_numpy(img), 'mask': torch.from_numpy(mask)}
    
# Customized Dataset
class ProstateCancerDataset(Dataset):
    """Prostate Cancer Biopsy Dataset"""
    
    def __init__(self, csv_file, root_dir, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file
            root_dir (string): Path to the directory with all images
            transform (callable, optional): Optional transform to be applied on an image sample
        """
        # Shuffle dataframes with fixed seed; otherwise, validation set only get cancerous samples
        self.cancer_df = pd.read_csv(csv_file).sample(frac=1, random_state=1)
        self.root_dir = root_dir
        self.transform = transform
        
    def __len__(self):
        return len(self.cancer_df)
    
    def __getitem__(self, idx):
        img_path = os.path.join(self.root_dir, f'{self.cancer_df.iloc[idx, 0]}_0.png')
        # (D,W,H)
        img = io.imread(img_path)
        image = tile_patriot(img, sz=128, N=16) # added new 
        img = cv2.hconcat([cv2.vconcat([image[0], image[1], image[2], image[3]]), 
                             cv2.vconcat([image[4], image[5], image[6], image[7]]), 
                             cv2.vconcat([image[8], image[9], image[10], image[11]]), 
                             cv2.vconcat([image[12], image[13], image[14], image[15]])])
        
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        isup = self.cancer_df.iloc[idx, 2]
        gleason = self.cancer_df.iloc[idx, 3]
        
        if self.transform:
            img = self.transform(img)
        sample = {'image': img, 'isup_grade': isup, 'gleason_score': gleason}
        return sample          
    

In [2]:
image_cropped_dir = '/project/yi_data/train_crop_128'
NUM_TRAIN = 6800 #8492

USE_GPU = True
dtype = torch.float32   # use float throughout the training
print_every = 100

SCALE_SZ = 128
BATCH_SZ = 8

if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
    
print('using device:', device)

using device: cuda


In [10]:
import cv2
import time
import copy
import torch
from torch.autograd import Variable
from utils import plot_training

ModuleNotFoundError: No module named 'torchnet'

In [5]:
def tile_patriot(img, sz=128, N=16):
    shape = img.shape
    pad0,pad1 = (sz - shape[0]%sz)%sz, (sz - shape[1]%sz)%sz
    img = np.pad(img,[[pad0//2,pad0-pad0//2],[pad1//2,pad1-pad1//2],[0,0]],
                 constant_values=255)
    img = img.reshape(img.shape[0]//sz,sz,img.shape[1]//sz,sz,3)
    img = img.transpose(0,2,1,3,4).reshape(-1,sz,sz,3)
    if len(img) < N:
        img = np.pad(img,[[0,N-len(img)],[0,0],[0,0],[0,0]],constant_values=255)
    idxs = np.argsort(img.reshape(img.shape[0],-1).sum(-1))[:N]
    img = img[idxs]
    return img#[N,size,size,3]

In [4]:
# Customized Dataset
class ProstateCancerDataset(Dataset):
    """Prostate Cancer Biopsy Dataset"""
    
    def __init__(self, csv_file, root_dir, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file
            root_dir (string): Path to the directory with all images
            transform (callable, optional): Optional transform to be applied on an image sample
        """
        # Shuffle dataframes with fixed seed; otherwise, validation set only get cancerous samples
        self.cancer_df = pd.read_csv(csv_file).sample(frac=1, random_state=1)
        self.root_dir = root_dir
        self.transform = transform
        
    def __len__(self):
        return len(self.cancer_df)
    
    def __getitem__(self, idx):
        img_path = os.path.join(self.root_dir, f'{self.cancer_df.iloc[idx, 0]}_0.png')
        # (D,W,H)
        img = io.imread(img_path)
        image = tile_patriot(img, sz=128, N=16) # added new 
        img = cv2.hconcat([cv2.vconcat([image[0], image[1], image[2], image[3]]), 
                             cv2.vconcat([image[4], image[5], image[6], image[7]]), 
                             cv2.vconcat([image[8], image[9], image[10], image[11]]), 
                             cv2.vconcat([image[12], image[13], image[14], image[15]])])
        
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        isup = self.cancer_df.iloc[idx, 2]
        gleason = self.cancer_df.iloc[idx, 3]
        
        if self.transform:
            img = self.transform(img)
        sample = {'image': img, 'isup_grade': isup, 'gleason_score': gleason}
        return sample        

##Customize Transforms
class Rescale(object):
    """Rescale the image sample to the given size
    Args:
        output_size (tuple): Desired output size. Output is matched to output_size.
    """
    
    def __init__(self, output_size):
        assert isinstance(output_size, tuple)
        self.output_size = output_size
        
    def __call__(self, sample):
        img = transform.resize(sample['image'], self.output_size)
        return {'image': img, 'isup_grade': sample['isup_grade'], 'gleason_score': sample['gleason_score']}
    
"""
class ToTensor(object):
    """Convert ndarrays in sample to Tensors."""
    
    def __call__(self, sample):
        img = sample['image']        
        # Swap color axis to [C,H,W]
        img = img.transpose(2,0,1)
        return {'image': img, 'isup_grade': sample['isup_grade'], 'gleason_score': sample['gleason_score']}
"""

# Data Preparation

In [7]:
# Compse tranforms of totensor
# More transformer to try: crop, normalize, etc.
# And transformer is a useful tool for data augmentation
biopsy_train = ProstateCancerDataset(csv_file='train_512.csv',
                                     root_dir=image_cropped_dir,
                                     transform=T.Compose([
                                                 T.ToTensor(),
                                                 T.Normalize((0.90960454,0.81946206,0.87811487),
                                                             (0.13244118,0.24944844,0.16392948)),
                                     ]))

loader_train = DataLoader(biopsy_train, batch_size=BATCH_SZ, num_workers=4,
                          sampler=sampler.SubsetRandomSampler(range(NUM_TRAIN)))

loader_val = DataLoader(biopsy_train, batch_size=BATCH_SZ, num_workers=4,
                        sampler=sampler.SubsetRandomSampler(range(NUM_TRAIN, 8492)))  #NUM_TRAIN+200  Use smaller size for debug

In [8]:
for i,k in enumerate(biopsy_train
                    ):
    print(i)
    print(k['image'].shape)
    break
    

0
torch.Size([3, 512, 512])


isup_distr = defaultdict(int)
for batch_i, batch_sample in enumerate(loader_train):
    #for grade in batch_sample['isup_grade']:
        #print(int(grade))
    #    isup_distr[int(grade)] += 1
    #print(batch_sample['image'][0].shape)
    #for s_i, sample in enumerate(batch_sample['isup_grade']):
    #    print(sample)
    #print(batch_sample['isup_grade'])
    #print(batch_sample['image'][0])
    print(batch_sample['image_id'][0])
    plt.imshow(batch_sample['image'][0].permute(1,2,0))
    assert False
print(isup_distr)

In [10]:
def train_model(model, dataloaders, criterion, optimizer, scheduler, num_epochs=5):
    since = time.time()
    
    train_acc_history = []
    val_acc_history = []
    loss_history = []
    
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    
    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs-1}')
        print('-'*10)
        
        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()   # Set model to training phase
            else:
                model.eval()    # Set model to evaluate phase
                
            running_loss = 0.0
            running_corrects = 0
            
            for batch in dataloaders[phase]:
                inputs = batch['image'].to(device=device, dtype=dtype)
                labels = batch['isup_grade'].to(device=device, dtype=torch.long)
                
                # Zero the parameter gradients
                optimizer.zero_grad()
                
                # Forward, track history if only in training
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)
                    
                    _, preds = torch.max(outputs, 1)

                    # Backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                    
                # Statistics
                running_loss += loss.item() * inputs.size(0)
                #print(loss.item())
                running_corrects += torch.sum(preds == labels)
            
            # End of epoch
            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)
            print('{} Loss: {:4f} Acc: {:4f}'.format(phase, epoch_loss, epoch_acc))
            
            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'val':
                val_acc_history.append(epoch_acc)
            else:
                train_acc_history.append(epoch_acc)
                loss_history.append(epoch_loss)
                
            if scheduler is not None and phase == 'train':
                scheduler.step()
                
        print()
    
    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:0f}s'.format(time_elapsed//60, time_elapsed%60))
    print('Best val Acc: {:4f}'.format(best_acc))
    
    model.load_state_dict(best_model_wts)
    return model, loss_history, train_acc_history, val_acc_history


def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False
            

def initialize_model(num_classes, feature_extract=False, use_pretrained=False):
    model_ft = None
    input_size = 0
    
    model_ft = models.alexnet(pretrained=use_pretrained)
    set_parameter_requires_grad(model_ft, feature_extract)
    num_ftrs = model_ft.classifier[6].in_features
    model_ft.classifier[6] = nn.Linear(num_ftrs, num_classes)
    #input_size = 128
    
    return model_ft, input_size

# Two-Layer Network

In [ ]:
def flatten(x):
    N = x.shape[0] # read in N, C, H, W
    return x.view(N, -1)

class Flatten(nn.Module):
    def forward(self, x):
        return flatten(x)
    
hidden_layer_size = 100
learning_rate = 4e-4

model_fc = nn.Sequential(
    Flatten(),
    nn.Linear(3*SCALE_SZ*SCALE_SZ, hidden_layer_size),
    nn.ReLU(),
    nn.Linear(hidden_layer_size, 6)
)

# Send the model to GPU/CPU
model_fc = model_fc.to(device)

# Use Nesterov momentum
optimizer = optim.SGD(model_fc.parameters(),
                      lr=learning_rate,
                      momentum=.9,
                      nesterov=True)
"""
optimizer = optim.Adam(model.parameters(),
                       lr=learning_rate)
"""
# Learning Rate scheduler
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.5)
best_model, loss_history, train_acc_history, val_acc_history = train_model(model_fc, {'train': loader_train, 'val': loader_val}, F.cross_entropy, optimizer, None, 30)

#train_sequential(model, optimizer, scheduler, 30)

In [ ]:
def flatten(x):
    N = x.shape[0] # read in N, C, H, W
    return x.view(N, -1)

class Flatten(nn.Module):
    def forward(self, x):
        return flatten(x)
    
hidden_layer_size = 100
learning_rate = 4e-4

model_fc = nn.Sequential(
    Flatten(),
#     nn.Linear(3*SCALE_SZ*SCALE_SZ, hidden_layer_size),
    nn.Linear(3*512*512, hidden_layer_size),
    nn.ReLU(),
    nn.Linear(hidden_layer_size, 6)
)

# Send the model to GPU/CPU
model_fc = model_fc.to(device)

# Use Nesterov momentum
optimizer = optim.SGD(model_fc.parameters(),
                      lr=learning_rate,
                      momentum=.9,
                      nesterov=True)
"""
optimizer = optim.Adam(model.parameters(),
                       lr=learning_rate)
"""
# Learning Rate scheduler
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.5)
best_model, loss_history, train_acc_history, val_acc_history = train_model(model_fc, {'train': loader_train, 'val': loader_val}, F.cross_entropy, optimizer, None, 30)

#train_sequential(model, optimizer, scheduler, 30)

In [ ]:
plt.subplot(2,1,1)
plt.plot(loss_history, 'o')
plt.xlabel('epoch')
plt.ylabel('loss')

plt.subplot(2,1,2)
plt.plot(train_acc_history, '-o')
plt.plot(val_acc_history, '-o')
plt.legend(['train', 'val'], loc='upper left')
plt.xlabel('epoch')
plt.ylabel('accuracy')
plt.show()

# CNN
Reference: https://pytorch.org/tutorials/beginner/finetuning_torchvision_models_tutorial.html

In [ ]:
model_ft, input_size = initialize_model(6)
#print(model_ft)
# Send the model to GPU/CPU
model_ft = model_ft.to(device)

optimizer = optim.SGD(model_ft.parameters(),
                      lr=3e-3,
                      momentum=.9,
                      nesterov=True)

best_model, loss_history, train_acc_history, val_acc_history = train_model(model_ft, {'train': loader_train, 'val': loader_val}, F.cross_entropy, optimizer, None, 30)
print(loss_history)
print(train_acc_history)
print(val_acc_history)

In [ ]:
 """
Graphs
1. loss vs. iterations
2. Train/Validation accuracy along epoch
"""
plt.subplot(2,1,1)
plt.plot(loss_history, 'o')
plt.xlabel('epoch')
plt.ylabel('loss')

plt.subplot(2,1,2)
plt.plot(train_acc_history, '-o')
plt.plot(val_acc_history, '-o')
plt.legend(['train', 'val'], loc='upper left')
plt.xlabel('epoch')
plt.ylabel('accuracy')
plt.show()

In [ ]:
print(best_model)